In [45]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [46]:
def data_filter(dat):
    dat['date'] = dat['date'].apply(lambda x: x.split('+')[0])
    dat['date'] = pd.to_datetime(dat['date']) # converting the date column to datetime format
    dat = dat[dat['date'] > '2017-12-31' ]
    dat = dat[dat['date'] < '2021-08-01' ]
    return dat

def is_body_size_greater_than_nx_average(df, n1, n2, window):
    
    df['candle_body'] = abs(df['close'] - df['open'])
    df['avg'] = df['candle_body'].rolling(window=window).mean()

    return ((df['candle_body'] > df['avg']*n1) & (df['candle_body'] < df['avg']*n2) ) & (df['open']>df['close'])

def add_imbalance_zone(dat):
    
    dat.loc[dat["is_imbalance"], "upper"] = np.minimum( dat.loc[dat["is_imbalance"], "min_last_30"], dat.loc[dat["is_imbalance"], "open"])  
    dat.loc[dat["is_imbalance"], "lower"] = dat.shift(-1).loc[dat["is_imbalance"], "high"]
    dat["zone"] = ((dat["upper"] - dat["lower"])>0) & ((dat["upper"] - dat["lower"]) > 0.2* abs(dat['open'] - dat['close']))
    
    return dat

In [47]:
245680

In [48]:
def buySignalFinder(df):
    df["checkSwing"] =  (df['high'].shift(-1) > df['high']) & (df['high'].shift(-1) > df['high'].shift(-2)) & (df['low'].shift(-1) > df['low']) & (df['low'].shift(-1) > df['low'].shift(-2))
    df["index"] = (df["checkSwing"].astype(int))*(df["high"])
    df["index"] = df['index'].where(df['index'] != 0, np.nan)
    df["index"].fillna(method="ffill",inplace=True)
    df["pivotHi"] = df["index"].fillna(0)
    df["Signal"] = (df["close"]>df["pivotHi"]) & df["pivotHi"]
    df['new']  = df["pivotHi"] * (df["Signal"].astype(int))
    duplicates = df['new'].duplicated()
    df.loc[duplicates, 'new'] = 0
    df['buy_signal'] = df['new'].where(df['new'] != 0, False).where(df['new'] ==0, True) 
    
    df.drop(columns = ["Signal","pivotHi","index","checkSwing","new",'close','high','low','open','volume_per_candle','volume'],inplace = True)
    return df


In [49]:
def resample_to_constant_volume_flex(data):
    
    def volumeSum_threshold():
        cumsum = 0
        val = yield cumsum
        while True:
            if (cumsum < val[1] ) or ("15:29" in val[2]) and ("09:15" not in val[2]):
                cumsum += val[0]
                if ("15:29" in val[2]):
                    cumsum += val[1]
                val = yield cumsum
            else:
                cumsum = val[0]
                val = yield cumsum
    a = volumeSum_threshold()
    next(a)
    data["volume_per_candle"] = data["volume"].rolling(window=360).mean().rolling(window = 100).quantile(0.90)
    data["volume_per_candle"].fillna(method="bfill",inplace=True)
    data["cumsum"] = data["volume"].cumsum()

    data["volume_vol"] = [a.send(v) for v in np.array(data[["volume","volume_per_candle","date"]])]
    
    data["thesholdBreak"] = data['volume_vol']>data["volume_per_candle"]
    
    data["close_vol"] = data["close"]*data["thesholdBreak"].astype(int)
    data["open_vol"] = data["open"]*((data["thesholdBreak"]).astype(int).shift(1))
    data["open_vol"].iloc[0] = data["open"].iloc[0]
    data["open_vol"] = data['open_vol'].where(data["open_vol"] != 0, np.nan)
    data["open_vol"].fillna(method="ffill",inplace=True)
    data["close_vol"] = data['close_vol'].where(data["close_vol"] != 0, np.nan)
    data["close_vol"].iloc[-1] = data['close'].iloc[-1]
    data["close_vol"].fillna(method="bfill",inplace=True)
    
    def high_():
        hh = 0
        val = yield hh
        while True:
            if val[0]:
                hh = max(hh,val[1])
                val = yield hh
                hh = 0
            else:
                hh = max(hh,val[1])
                val = yield hh
    h = high_()
    next(h)
    def low_():
        ll = np.inf
        val = yield ll
        while True:
            if val[0]:
                ll = min(ll,val[1])
                val = yield ll
                ll = np.inf
            else:
                ll = min(ll,val[1])
                val = yield ll
    l = low_()
    next(l)
    
    data["high_vol"]=[h.send(v) for v in np.array(data[["thesholdBreak","high"]])]
    data["low_vol"] =[l.send(v) for v in np.array(data[["thesholdBreak","low"]])]
    data["open"] = data["open_vol"]
    data["high"] = data["high_vol"]
    data["low"] = data["low_vol"]
    data["close"] = data["close_vol"]
    data["volume"] = data["volume_vol"]
    
    
    return data[["date","open","high","low","close","volume_per_candle","volume"]][data["thesholdBreak"]]

In [50]:
dat = pd.read_csv('ACC_minute_data_with_indicators.csv', usecols = ['date' , 'open' , 'high', 'low' , 'close' , 'volume'] )
dat = data_filter(dat)
dat['is_imbalance'] = is_body_size_greater_than_nx_average(dat,3,5, 30)
dat['min_last_30'] = dat['low'].shift(1).rolling(window=30).min()
dat = add_imbalance_zone(dat)
dat = is_still_tradable(dat)
dat["ema_100"] = np.round(dat["close"].ewm(100).mean(),2)
pd.options.display.max_rows = 10000
dat['date'] = dat['date'].astype(str)
dat_resample = resample_to_constant_volume_flex(dat.copy())
merged_df = pd.merge(dat, buySignalFinder(dat_resample), on = 'date', how = 'left').fillna(False)
merged_df['buy_PA'] = merged_df['still_tradable'] * merged_df['buy_signal']

In [51]:
index = [171, 233, 340, 342, 351, 352, 1570, 1780]

In [54]:
merged_df

,level_0,index,date,close,high,low,open,volume,candle_body,avg,is_imbalance,min_last_30,upper,lower,zone,lowUse,still_tradable,ema_100,buy_signal,buy_PA
0,0,249971,2018-01-01 09:15:00,1760.00,1763.90,1758.65,1763.90,718,3.90,False,False,False,False,False,False,inf,False,1760.00,False,False
1,1,249972,2018-01-01 09:16:00,1763.80,1763.80,1760.00,1760.00,409,3.80,False,False,False,False,False,False,inf,False,1761.91,False,False
2,2,249973,2018-01-01 09:17:00,1760.10,1763.45,1760.00,1763.45,867,3.35,False,False,False,False,False,False,inf,False,1761.30,False,False
3,3,249974,2018-01-01 09:18:00,1759.35,1760.45,1759.35,1760.10,1065,0.75,False,False,False,False,False,False,inf,False,1760.81,False,False
4,4,249975,2018-01-01 09:19:00,1758.65,1759.90,1758.65,1759.35,436,0.70,False,False,False,False,False,False,inf,False,1760.37,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304699,304699,554670,2021-07-28 15:25:00,2391.70,2396.75,2390.15,2395.35,2622,3.65,0.901667,True,2388.5,2388.5,2395.35,False,inf,False,2375.17,False,False
304700,304700,554671,2021-07-28 15:26:00,2395.00,2395.35,2391.70,2391.70,2092,3.30,0.995,False,2388.5,False,False,False,inf,False,2375.37,False,False
304701,304701,554672,2021-07-28 15:27:00,2391.15,2395.00,2391.15,2394.95,1189,3.80,1.08167,True,2388.5,2388.5,2394.6,False,inf,False,2375.53,False,False
304702,304702,554673,2021-07-28 15:28:00,2386.10,2394.60,2384.80,2391.15,3110,5.05,1.25,True,2388.5,2388.5,2394,False,inf,False,2375.63,False,False


In [55]:
merged_df.buy_PA.sum()

75

In [56]:
merged_df.is_imbalance.sum()

5487

In [57]:
merged_df.zone.sum()

1086

In [58]:
merged_df.still_tradable.sum()

3620

In [59]:
merged_df.buy_signal.sum()

5051

In [43]:
merged_df.drop(columns = ['candle_body','avg','min_last_30','ema_100','buy_signal'], inplace = True)

In [16]:
merged_df[merged_df['is_imbalance'] == True and merged_df['zone'] == True]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().